In [ ]:
# Importar las bibliotecas necesarias
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from statsmodels.tsa.stattools import adfuller
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping
import warnings

warnings.filterwarnings("ignore")  # Ignorar advertencias

In [ ]:
# Subir el archivo desde el sistema local
from google.colab import files
uploaded = files.upload()  # Abre un cuadro de diálogo para subir archivos

# Leer el archivo CSV en un DataFrame
df = pd.read_csv('Enso.csv')  # Asegúrate de que el nombre del archivo coincida

Saving Enso.csv to Enso.csv


In [ ]:
# Convertir la columna de fecha en un objeto de fecha (si es necesario)
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Establecer la columna de fecha como índice si está presente
if 'Date' in df.columns:
    df.set_index('Date', inplace=True)

# Visualizar las primeras filas del DataFrame para entender su estructura
print("Primeras filas del DataFrame:")
print(df.head())

Primeras filas del DataFrame:
            nino12  nino3  nino4  nino34
Date                                    
1870-01-01   -1.50  -1.35  -0.48   -1.00
1870-02-01   -0.96  -1.20  -1.16   -1.20
1870-03-01   -0.65  -0.76  -1.01   -0.83
1870-04-01   -0.32  -0.78  -1.00   -0.81
1870-05-01   -0.64  -1.15  -1.08   -1.27


In [ ]:
# Definir una función para crear conjuntos de datos de secuencias
def create_dataset(series, time_steps=1):
    X, y = [], []
    for i in range(len(series) - time_steps):
        X.append(series[i:(i + time_steps)])
        y.append(series[i + time_steps])
    return np.array(X), np.array(y)

# Definir una función para ajustar un modelo CNN y devolver los estadísticos de predicción
def cnn_forecast(series, time_steps=10, epochs=50, batch_size=32):
    # Asegurarse de que la serie sea un vector unidimensional (sin nulos)
    series = series.dropna().astype(float)

    # Normalizar la serie de tiempo
    scaler = MinMaxScaler(feature_range=(0, 1))
    series_scaled = scaler.fit_transform(series.values.reshape(-1, 1))

    # Dividir los datos en conjunto de entrenamiento (70%) y prueba (30%)
    train_size = int(len(series_scaled) * 0.7)
    train, test = series_scaled[:train_size], series_scaled[train_size:]

    # Crear conjuntos de datos de secuencias
    X_train, y_train = create_dataset(train, time_steps)
    X_test, y_test = create_dataset(test, time_steps)

    # Remodelar los datos para ser compatibles con Conv1D
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    # Ajustar el modelo CNN
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Añadir early stopping para evitar el sobreajuste
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Entrenar el modelo
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), callbacks=[early_stopping], verbose=0)

    # Generar pronósticos
    forecast = model.predict(X_test)
    forecast = scaler.inverse_transform(forecast)

    # Invertir la escala de los datos de prueba
    y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Calcular métricas de evaluación
    mse = mean_squared_error(y_test, forecast)  # Error Cuadrático Medio
    rmse = np.sqrt(mse)  # Raíz del Error Cuadrático Medio
    mae = mean_absolute_error(y_test, forecast)  # Error Absoluto Medio
    r2 = r2_score(y_test, forecast)  # Coeficiente de Determinación

    # Calcular MAPE manejando valores cero
    mape = np.mean(np.abs((y_test - forecast) / (y_test + np.finfo(float).eps))) * 100  # Evitar división por cero

    # Realizar la prueba de Dickey-Fuller aumentada para la estacionariedad
    adf_test = adfuller(series)
    adf_statistic = adf_test[0]  # Estadístico ADF
    adf_p_value = adf_test[1]  # Valor p de la prueba ADF

    # Devolver las métricas y los estadísticos en un diccionario
    return {
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'mape': mape,
        'adf_statistic': adf_statistic,
        'adf_p_value': adf_p_value
    }

# Aplicar el modelo CNN a cada una de las series de tiempo numéricas en el DataFrame
cnn_results = {}
for column in df.select_dtypes(include=[np.number]).columns:
    print(f"Procesando la serie de tiempo: {column}")
    cnn_results[column] = cnn_forecast(df[column])

# Crear un DataFrame para almacenar los resultados
results_df = pd.DataFrame(cnn_results).T  # Transponer para una mejor presentación
results_df.columns = ['MSE', 'RMSE', 'MAE', 'R²', 'MAPE', 'ADF Statistic', 'ADF P-value']  # Renombrar columnas

# Mostrar el DataFrame con los resultados
print("Resumen de los resultados de CNN:")
print(results_df)

# Guardar el DataFrame en un archivo CSV
results_df.to_csv('cnn_model_comparison.csv', index=True)  # Guardar el DataFrame en un archivo CSV
files.download('cnn_model_comparison.csv')  # Descargar el archivo CSV generado

Procesando la serie de tiempo: nino12
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Procesando la serie de tiempo: nino3
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Procesando la serie de tiempo: nino4
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Procesando la serie de tiempo: nino34
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Resumen de los resultados de CNN:
             MSE      RMSE       MAE        R²          MAPE  ADF Statistic  \
nino12  0.208741  0.456882  0.360425  0.817221  1.168967e+15     -11.532023   
nino3   0.093605  0.305949  0.248134  0.885068  7.561086e+14     -10.225140   
nino4   0.036054  0.189878  0.152943  0.910134  2.135156e+14      -9.779188   
nino34  0.078103  0.279470  0.226140  0.896661  4.651278e+14     -10.125987   

         ADF P-value  
nino12  3.811493e-21  
nino3   5.200632e-18  
nino4   6.805263e-17  
nino34  9.179166e-18  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>